# Merge for second supplement

So, I've got metadata tables listing books that will
supplement the central, random sample in a variety of ways.

I need to merge those tables into a single master list that
has all the information we need for further analysis.

In theory this could be done with joins, but it would end
up being opaque both to me and to readers. Better in this
case, I think, to spell everything out with procedural
explicitness.

In [73]:
import pandas as pd
from difflib import SequenceMatcher

In [2]:
work = pd.read_csv('../../noveltmmeta/workmeta.tsv', sep = '\t', low_memory = False, index_col = 'docid')
work.columns.tolist()

['oldauthor',
 'author',
 'authordate',
 'inferreddate',
 'latestcomp',
 'datetype',
 'startdate',
 'enddate',
 'imprint',
 'imprintdate',
 'contents',
 'genres',
 'subjects',
 'geographics',
 'locnum',
 'oclc',
 'place',
 'recordid',
 'instances',
 'allcopiesofwork',
 'copiesin25yrs',
 'enumcron',
 'volnum',
 'title',
 'parttitle',
 'earlyedition',
 'shorttitle']

In [3]:
best = pd.read_csv('bestsellers/deduplicated_bestsellers.tsv', sep = '\t')

In [4]:
matched = pd.read_csv('canon/fuzzy_matched_canon.tsv', sep = '\t')

In [7]:
firstsample = pd.read_csv('../supplement1/lastnamesupp1meta.tsv', sep = '\t')

In [48]:
master = pd.read_csv('../../noveltmmeta/masterficmetadata.tsv', sep = '\t', low_memory = False, index_col = 'docid')

### Desired columns

    docid
    author
    title -- will be equiv to workmeta "shorttitle"
    inferreddate (workmeta)
    latestcomp (workmeta -- this will be used for processing, and must be filled out)
    firstpub (a step beyond latestcomp to true firstpub; we won't have for all rows)
    allcopiesofwork
    copiesin25yrs
    gender
    nationality
    earlyedition
    lastname
    imprint
    
In addition to general metadata, there will be several columns that define reasons for selection. Since the same volume can be selected for more than one reason, it's possible to have "True" in more than one of these columns.

    heath
    norton
    nortonshort
    bestseller
    mostdiscussed
    reviewed
    contrast4reviewed
    preregistered
    


In [55]:
newdata = dict()

In [56]:
for idx, row in firstsample.iterrows():
    r = dict()
    r['docid'] = row['docid']
    r['author'] = row['author']
    r['title'] = row['title']
    r['inferreddate'] = row['actualdate']
    r['latestcomp'] = row['earliestdate']
    r['firstpub'] = row['earliestdate']
    r['allcopiesofwork'] = float('nan')
    r['copiesin25yrs'] = float('nan')
    r['gender'] = row['gender']
    r['nationality'] = row['nationality']
    r['earlyedition'] = float('nan')
    r['lastname'] = row['lastname']
    r['imprint'] = float('nan')
    
    if row['tags'] == 'vulgar':
        r['contrast4reviewed'] = True
        r['reviewed'] = False
        r['preregistered'] = False
    elif row['tags'] == 'elite':
        r['contrast4reviewed'] = False
        r['reviewed'] = True
        r['preregistered'] = False
    elif row['tags'] == 'special':
        r['preregistered'] = True
        r['reviewed'] = False
        r['contrast4reviewed'] = False
    else:
        continue
    
    for col in ['mostdiscussed', 'bestseller', 'nortonshort', 'norton', 'heath']:
        r[col] = False
        # for now! this could be revised later
    
    newdata[r['docid']] = r    

In [57]:
for idx, row in best.iterrows():
    docid = row['docid']
    if docid in newdata:
        r = newdata[docid]
        r['imprint'] = row['imprint']
        
        if pd.isnull(row['firstpub']):
            firstpub = float('nan')
        else:
            firstpub = int(row['firstpub'])
            
        if firstpub < int(r['latestcomp']):
            r['latestcomp'] = row['firstpub']
        
    else:
        r = dict()
        r['docid'] = row['docid']
        r['author'] = row['author']
        r['title'] = row['title']
        r['inferreddate'] = row['inferreddate']
        r['latestcomp'] = row['firstpub']
        r['firstpub'] = row['firstpub']
        r['allcopiesofwork'] = float('nan')
        r['copiesin25yrs'] = float('nan')
        r['gender'] = row['gender']
        r['nationality'] = row['nationality']
        r['earlyedition'] = float('nan')
        r['lastname'] = float('nan')
        r['imprint'] = float('nan')
        for col in ['mostdiscussed', 'nortonshort', 'norton', 'heath', 'preregistered', 
                    'reviewed', 'contrast4reviewed', 'bestseller']:
            r[col] = False
    
    r['bestseller'] = True
    newdata[docid] = r

In [58]:
obligatecols = ['docid', 'author', 'title', 'inferreddate', 'latestcomp', 'firstpub', 
                'allcopiesofwork', 'copiesin25yrs', 'gender', 'nationality', 'earlyedition',
               'lastname', 'imprint']
tfcols = ['norton', 'heath', 'nortonshort', 'mostdiscussed', 'preregistered', 'reviewed', 'contrast4reviewed', 'bestseller']

In [59]:
for idx, row in matched.iterrows():
    r = dict()
    r['docid'] = row['docid']
    r['author'] = row['author']
    r['title'] = row['shorttitle']
    r['inferreddate'] = row['inferreddate']
    r['latestcomp'] = row['latestcomp']
    r['firstpub'] = float('nan')
    r['allcopiesofwork'] = row['allcopiesofwork']
    r['copiesin25yrs'] = row['copiesin25yrs']
    r['gender'] = float('nan')
    r['nationality'] = float('nan')
    r['earlyedition'] = row['earlyedition']
    r['lastname'] = float('nan')
    r['imprint'] = row['imprint']
    r['norton'] = row['norton']
    r['heath'] = row['heath']
    r['nortonshort'] = row['nortonshort']
    r['mostdiscussed'] = False
    r['preregistered'] = False
    r['reviewed'] = False
    r['contrast4reviewed'] = False
    r['bestseller'] = False

    if r['docid'] in newdata:
        toupdate = newdata[r['docid']]
        
        for col in obligatecols:
            if pd.isnull(toupdate[col]) and not pd.isnull(r[col]):
                toupdate[col] = r[col]
                print(toupdate[col])
        
        for col in tfcols:
            if toupdate[col] == False and r[col] == True:
                toupdate[col] = True
        
        newdata[r['docid']] = toupdate
    
    else:
        newdata[r['docid']] = r
    

18
2
True
25
4
True
Boston;Ticknor and Field;1866.
1885
31
4
True
New York;C. L. Webster and company;1885.
1906
8
2
True
New York;Doubleday, Page & Company;1906.
1912
2
2
True
Boston;New York;Houghton Mifflin Co.;1912.
1917
7
3
True
New York;Grosset & Dunlap;c1917.
1939
11
6
True
New York|The Viking press|c1939
1952
5
2
True
New York|Random House|1952


In [60]:
len(newdata)

2121

In [62]:
correx = 0
caught = 0

for docid, row in newdata.items():
    if pd.isnull(row['lastname']):
        if pd.isnull(row['author']):
            row['lastname'] == 'anon'
        else:
            names = row['author'].split()
            lastname = names[0].strip('()[],.')
            row['lastname'] = lastname
            
    if docid in work.index:
        refrow = work.loc[docid, : ]
        for col in obligatecols:
            if col in {'firstpub', 'gender', 'nationality', 'lastname'}:
                continue
                
            if pd.isnull(row[col]) and not pd.isnull(refrow[col]):
                row[col] = refrow[col]
                correx += 1
    else:
        if docid in master.index and pd.isnull(row['imprint']):
            row['imprint'] = master.loc[docid, 'imprint']
            caught += 1
            
print(correx)
print(caught)
            

6307
455


In [63]:
aslist = []
for key, value in newdata.items():
    aslist.append(value)
newdf = pd.DataFrame(aslist)
newdf.head()

,allcopiesofwork,author,bestseller,contrast4reviewed,copiesin25yrs,docid,earlyedition,firstpub,gender,heath,...,inferreddate,lastname,latestcomp,mostdiscussed,nationality,norton,nortonshort,preregistered,reviewed,title
0,3.0,"Hough, Emerson,",True,False,3.0,mdp.39015063923869,True,1909.0,NaN,False,...,1909.0,Hough,1909.0,False,NaN,False,False,False,False,54-40 or fight
1,27.0,"Cooper, James Fenimore",False,False,3.0,uc1.b3321331,True,NaN,NaN,False,...,1856.0,Cooper,1851.0,False,NaN,True,False,False,False,The last of the Mohicans : a narrative of 1757
2,1.0,anonymous6,False,False,1.0,uc1.32106006691585,True,1902.0,f,False,...,1902.0,anonymous6,1902.0,False,ir,False,False,False,True,Cuchulain of Muirthemne
3,NaN,"Scott, Walter,",True,False,NaN,nyp.33433075743603,NaN,1818.0,m,False,...,1824.0,Scott,1818.0,False,uk,False,False,False,False,Rob Roy
4,NaN,"Williams, Ben Ames",False,False,NaN,mdp.39015030719481,NaN,1944.0,m,False,...,1944.0,Williams,1944.0,False,us,False,False,False,True,Leave Her to Heaven


In [64]:
cols = list(obligatecols)
cols.extend(tfcols)
newdf = newdf[cols]
newdf.to_csv('second_supplement.tsv', sep = '\t', index = False)

In [65]:
sum(newdf.bestseller)

849

In [66]:
sum(newdf.reviewed)

600

In [68]:
sum(newdf.heath)

46

In [69]:
discussed = pd.read_csv('canon/most_discussed.tsv', sep = '\t')

In [70]:
discussed.head()

,docid,author,title
0,NaN,"James, Henry",The American
1,X,"James, Henry",The Turn of the Screw
2,NaN,"James, Henry",Portrait of a Lady
3,NaN,"Melville, Herman",Moby Dick
4,X,"Morrison, Toni",Beloved


In [71]:
def blockcode(aname):
    if pd.isnull(aname):
        block = 'xx'
    elif len(aname) < 3:
        block = 'nn'
    else:
        block = aname.lower()[0:2]
    
    return block

work = work.assign(block = work.author.map(blockcode))

byblock = work.groupby('block')
block_dictionary = dict()

for code, df in byblock:
    block_dictionary[code] = df

In [76]:
def interpret_tag(volnum):
    try:
        volnum = int(volnum)
    except:
        volnum = 1
        print('error in source')
        
    if volnum == 1:
        return 'norton'
    elif volnum == 2:
        return 'nortonshort'
    elif volnum == 3:
        return 'heath'
    else:
        print('This should never happen.')

def fuzzymatch(str1, str2):
    
    m = SequenceMatcher(None, str1, str2)
    match = m.real_quick_ratio()
    if match > 0.7:
        match = m.ratio()
    
    return match

allmatches = []

for idx, row in discussed.iterrows():
    if pd.isnull(row['docid']):
        continue
        
    normname = row['author'].lower()
    
    title = row['title'].lower()
    if len(title) > 28:
        title = title[0:28]
    code = normname[0:2]
    
    possiblematches = []
    
    if code in block_dictionary:
        block = block_dictionary[code]
        for idx2, row2 in block.iterrows():
            auth2match = row2['author'].lower()
            if '(' in auth2match:
                auth2match = auth2match.split('(')[0]
            authsimilarity = fuzzymatch(normname, auth2match)
            if authsimilarity < 0.74: 
                continue
            
            title2match = row2['shorttitle']
            if pd.isnull(title2match):
                title2match = 'x x x'
            else:
                title2match = title2match.lower()
                if len(title2match) > 28:
                    title2match = title2match[0:28]
                
            titlesimilarity = fuzzymatch(title, title2match)
            if titlesimilarity > 0.7:
                possiblematches.append((authsimilarity * titlesimilarity, idx2))

    if len(possiblematches) > 0:
        possiblematches.sort()
        probability, matchidx = possiblematches[-1]
        
        print(normname, ' | ', title, block.loc[matchidx, 'author'], ' == ', block.loc[matchidx, 'shorttitle'])
        allmatches.append(matchidx)
    else:
        print(normname, ' || ', title, "NO MATCH")

james, henry  |  the turn of the screw James, Henry  ==  The turn of the screw : and other stories
morrison, toni  |  beloved Morrison, Toni  ==  Love
morrison, toni  |  sula Morrison, Toni  ==  Sula
james, henry  |  ambassadors James, Henry  ==  The ambassadors
nabokov, vladimir  ||  ada NO MATCH
james, henry  |  wings of the dove James, Henry  ==  The wings of the dove
stowe, harriet beecher  |  dred Stowe, Harriet Beecher  ==  Dred
james, henry  |  golden bowl James, Henry  ==  The golden bowl
faulkner, william  |  sanctuary Faulkner, William  ==  Sanctuary
morrison, toni  |  jazz Morrison, Toni  ==  Jazz
melville, herman  ||  confidence man NO MATCH
hurston, zora neale  |  their eyes were watching god Hurston, Zora Neale  ==  Their eyes were watching God; a novel
hawthorne, nathaniel  |  the house of the seven gable Hawthorne, Nathaniel  ==  The house of the seven gables : and The snow image and other Twice-told tales--
hawthorne, nathaniel  |  the blithedale romance Hawthorne, Nat

In [77]:
allmatches

['pst.000029895397',
 'mdp.39015059959802',
 'mdp.39015043798431',
 'coo.31924052984741',
 'uc2.ark+=13960=fk8pc2tq9r',
 'uc1.b3325140',
 'uva.x000926858',
 'inu.30000048939361',
 'uc1.32106016923267',
 'uc1.32106007902726',
 'uva.x030497933',
 'nyp.33433076087844',
 'mdp.39015046349141',
 'uc2.ark+=13960=t5j96177f',
 'mdp.39015040129580']

In [79]:
takecols = ['author', 'shorttitle', 'inferreddate', 'latestcomp',
                'allcopiesofwork', 'copiesin25yrs', 'earlyedition', 'imprint']
discussdf = work.loc[allmatches, takecols]
discussdf.head()

,author,shorttitle,inferreddate,latestcomp,allcopiesofwork,copiesin25yrs,earlyedition,imprint
docid,,,,,,,,
pst.000029895397,"James, Henry",The turn of the screw : and other stories,1969,1916,1,1,False,Harmondsworth|Penguin|1969.
mdp.39015059959802,"Morrison, Toni",Love,2003,2003,1,1,True,New York|Knopf|Distributed by Random House|c2003.
mdp.39015043798431,"Morrison, Toni",Sula,1973,1973,4,3,True,"New York, NY|Plume|c1973."
coo.31924052984741,"James, Henry",The ambassadors,1903,1903,17,5,True,New York and London;Harper & brothers;1903.
uc2.ark+=13960=fk8pc2tq9r,"James, Henry",The wings of the dove,1902,1902,23,16,True,Westminster;A. Constable and co. ltd.;1902.


In [80]:
discussdf['gender'] = float('nan')
discussdf['firstpub'] = float('nan')
discussdf['nationality'] = float('nan')

def get_lastname(aname):
    names = aname.split()
    lastname = names[0].strip(',')
    return lastname

discussdf = discussdf.assign(lastname = discussdf.author.map(get_lastname))
discussdf.head()

,author,shorttitle,inferreddate,latestcomp,allcopiesofwork,copiesin25yrs,earlyedition,imprint,gender,firstpub,nationality,lastname
docid,,,,,,,,,,,,
pst.000029895397,"James, Henry",The turn of the screw : and other stories,1969,1916,1,1,False,Harmondsworth|Penguin|1969.,NaN,NaN,NaN,James
mdp.39015059959802,"Morrison, Toni",Love,2003,2003,1,1,True,New York|Knopf|Distributed by Random House|c2003.,NaN,NaN,NaN,Morrison
mdp.39015043798431,"Morrison, Toni",Sula,1973,1973,4,3,True,"New York, NY|Plume|c1973.",NaN,NaN,NaN,Morrison
coo.31924052984741,"James, Henry",The ambassadors,1903,1903,17,5,True,New York and London;Harper & brothers;1903.,NaN,NaN,NaN,James
uc2.ark+=13960=fk8pc2tq9r,"James, Henry",The wings of the dove,1902,1902,23,16,True,Westminster;A. Constable and co. ltd.;1902.,NaN,NaN,NaN,James


In [81]:
for c in tfcols:
    if c == 'mostdiscussed':
        discussdf[c] = True
    else:
        discussdf[c] = False

In [83]:
discussdf = discussdf.rename(index = str, columns = {'shorttitle': 'title'})
discussdf.head()

,author,title,inferreddate,latestcomp,allcopiesofwork,copiesin25yrs,earlyedition,imprint,gender,firstpub,nationality,lastname,norton,heath,nortonshort,mostdiscussed,preregistered,reviewed,contrast4reviewed,bestseller
docid,,,,,,,,,,,,,,,,,,,,
pst.000029895397,"James, Henry",The turn of the screw : and other stories,1969,1916,1,1,False,Harmondsworth|Penguin|1969.,NaN,NaN,NaN,James,False,False,False,True,False,False,False,False
mdp.39015059959802,"Morrison, Toni",Love,2003,2003,1,1,True,New York|Knopf|Distributed by Random House|c2003.,NaN,NaN,NaN,Morrison,False,False,False,True,False,False,False,False
mdp.39015043798431,"Morrison, Toni",Sula,1973,1973,4,3,True,"New York, NY|Plume|c1973.",NaN,NaN,NaN,Morrison,False,False,False,True,False,False,False,False
coo.31924052984741,"James, Henry",The ambassadors,1903,1903,17,5,True,New York and London;Harper & brothers;1903.,NaN,NaN,NaN,James,False,False,False,True,False,False,False,False
uc2.ark+=13960=fk8pc2tq9r,"James, Henry",The wings of the dove,1902,1902,23,16,True,Westminster;A. Constable and co. ltd.;1902.,NaN,NaN,NaN,James,False,False,False,True,False,False,False,False


In [85]:
newdf = pd.read_csv('second_supplement.tsv', sep = '\t', index_col = 'docid')
print(newdf.shape)
unified = pd.concat([newdf, discussdf])
unified.shape

(2121, 20)


(2136, 20)

In [88]:
unified.reset_index(inplace = True)
unified = unified[cols]
unified.to_csv('second_supplement_full.tsv', sep = '\t', index = False)

In [89]:
therandomsample = pd.read_csv('lastnamesample.tsv', sep = '\t')

In [90]:
overlap = set(unified.docid).intersection(set(therandomsample.docid))
print(len(overlap))

554
